## Applying LSTM Models on Raw Data

In [3]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

In [4]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

    
   # result = confusion_matrix(Y_true, Y_pred)

    #plt.figure(figsize=(10, 8))
   # sns.heatmap(result, 
    #            xticklabels= list(ACTIVITIES.values()), 
     #           yticklabels=list(ACTIVITIES.values()), 
      #          annot=True, fmt="d");
   # plt.title("Confusion matrix")
   # plt.ylabel('True label')
   # plt.xlabel('Predicted label')
    plt.show()  

#### Loading Data / Making Data

In [6]:
# Data directory
DATADIR = 'C:/Users/basan/Downloads/human+activity+recognition+using+smartphones (1)/UCI HAR Dataset/UCI HAR Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [14]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'C:/Users/basan/Downloads/human+activity+recognition+using+smartphones (1)/UCI HAR Dataset/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [15]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'C:/Users/basan/Downloads/human+activity+recognition+using+smartphones (1)/UCI HAR Dataset/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [16]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [17]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [18]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [19]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [20]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [21]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


#### 1. Defining the Architecture of 1-Layer of LSTM

In [22]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

C:\Users\basan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 32)                  │           5,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,574 (21.77 KB)

 Trainable params: 5,574 (21.77 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [24]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test),epochs=epochs)

Epoch 1/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.4333 - loss: 1.3682 - val_accuracy: 0.6193 - val_loss: 0.9597
Epoch 2/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.6532 - loss: 0.7912 - val_accuracy: 0.7011 - val_loss: 0.7407
Epoch 3/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.7387 - loss: 0.6322 - val_accuracy: 0.7961 - val_loss: 0.6301
Epoch 4/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.8295 - loss: 0.5011 - val_accuracy: 0.8314 - val_loss: 0.6027
Epoch 5/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.8679 - loss: 0.4161 - val_accuracy: 0.8660 - val_loss: 0.4485
Epoch 6/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9015 - loss: 0.3471 - val_accuracy: 0.8816 - val_loss: 0.3917
Epoch 7/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9098 - loss: 0.2909 - val_accuracy: 0.8728 - val_loss: 0.4303
Epoch 8/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9195 - loss: 0.2564 - val_accu

In [26]:
# Confusion Matrix
confusion_matrix(Y_test, model.predict(X_test))

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,378,113,0,0,0
STANDING,0,85,446,1,0,0
WALKING,0,0,0,472,19,5
WALKING_DOWNSTAIRS,0,0,0,4,409,7
WALKING_UPSTAIRS,0,0,0,6,6,459


In [27]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8679 - loss: 0.4527

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.3800520896911621, 0.9073634147644043]


##### With a simple 1 layer architecture we got 90.09% accuracy and a loss of 0.47

#### 2.1 First Model for 2-Layer of LSTM with more hyperparameter tunning

In [28]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(Y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [29]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 


C:\Users\basan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 128, 48)             │          11,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 48)             │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128, 48)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 32)                  │          10,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,894 (85.52 KB)

 Trainable params: 21,798 (85.15 KB)

 Non-trainable params: 96 (384.00 B)

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Epoch 1/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.5708 - loss: 1.9427 - val_accuracy: 0.7971 - val_loss: 1.0541
Epoch 2/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.8182 - loss: 0.9186 - val_accuracy: 0.8493 - val_loss: 0.6012
Epoch 3/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.8925 - loss: 0.4786 - val_accuracy: 0.9002 - val_loss: 0.3534
Epoch 4/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9164 - loss: 0.2884 - val_accuracy: 0.8870 - val_loss: 0.3554
Epoch 5/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9244 - loss: 0.2436 - val_accuracy: 0.9063 - val_loss: 0.2683
Epoch 6/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9301 - loss: 0.2016 - val_accuracy: 0.9203 - val_loss: 0.2077
Epoch 7/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - accuracy: 0.9286 - loss: 0.2009 - val_accuracy: 0.9148 - val_loss: 0.2668
Epoch 8/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - accuracy: 0.9374 - loss: 0.1747 - 

In [32]:
# Confusion Matrix
confusion_matrix(Y_test, model.predict(X_test))

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,5,379,106,0,0,1
STANDING,0,114,418,0,0,0
WALKING,0,0,0,445,50,1
WALKING_DOWNSTAIRS,0,0,0,0,418,2
WALKING_UPSTAIRS,0,0,0,0,46,425


In [33]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8390 - loss: 0.4877

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.4133538007736206, 0.8897183537483215]


### 2.2 Second Model for 2-Layer of LSTM with more hyperparameter tunning

In [34]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 


C:\Users\basan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 128, 64)             │          18,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 48)                  │          21,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             294 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,190 (160.90 KB)

 Trainable params: 41,062 (160.40 KB)

 Non-trainable params: 128 (512.00 B)

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Epoch 1/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.5961 - loss: 2.1089 - val_accuracy: 0.7099 - val_loss: 1.3926
Epoch 2/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - accuracy: 0.8310 - loss: 0.9904 - val_accuracy: 0.8948 - val_loss: 0.6529
Epoch 3/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.9086 - loss: 0.5397 - val_accuracy: 0.8979 - val_loss: 0.4508
Epoch 4/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.9190 - loss: 0.3270 - val_accuracy: 0.9114 - val_loss: 0.2912
Epoch 5/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.9328 - loss: 0.2254 - val_accuracy: 0.9121 - val_loss: 0.2657
Epoch 6/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 19s 42ms/step - accuracy: 0.9319 - loss: 0.1935 - val_accuracy: 0.9121 - val_loss: 0.2526
Epoch 7/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.9373 - loss: 0.1740 - val_accuracy: 0.9128 - val_loss: 0.2582
Epoch 8/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.9427 - loss: 0.1547 - 

In [37]:
# Confusion Matrix
confusion_matrix(Y_test, model.predict(X_test))

93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,9,410,72,0,0,0
STANDING,0,104,428,0,0,0
WALKING,0,0,0,477,18,1
WALKING_DOWNSTAIRS,0,0,0,0,418,2
WALKING_UPSTAIRS,0,0,0,10,14,447


In [38]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8717 - loss: 0.2964

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.2314060479402542, 0.9219545125961304]
